In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from scipy.signal import spectrogram


SCRIPT_PATH = os.getcwd()
INPUT_DIR = os.path.abspath(os.path.join(SCRIPT_PATH, '..', 'data', 'IntermediateData'))
OUTPUT_DIR = os.path.join(SCRIPT_PATH, '..', 'data', 'spectrograms')
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [2]:
def plot_spectrogram(signal, fs, title, save_path=None):
    """
    Generate and save spectrogram.
    """
    f, t, Sxx = spectrogram(signal, fs=fs, nperseg=256, noverlap=128)
    Sxx_db = 10 * np.log10(Sxx + 1e-10)  # log-scale (dB)

    plt.figure(figsize=(8, 4))
    plt.pcolormesh(t, f, Sxx_db, shading='gouraud')
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [s]')
    plt.title(title)
    plt.colorbar(label='Power [dB]')

    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        plt.close()
    else:
        plt.show()

In [3]:
def process_dataset(df, label, fs_estimate=1000):
    """
    Generate spectrogram for each experiment.
    """
    out_dir = os.path.join(OUTPUT_DIR, label)
    os.makedirs(out_dir, exist_ok=True)

    experiment_ids = df['Experiment ID'].unique()
    print(f"Generating spectrograms for {label} ({len(experiment_ids)} experiments)...")

    for exp_id in experiment_ids:
        tmp = df[df['Experiment ID'] == exp_id]
        time = tmp['Time (s)'].values
        current = tmp['CURRENT (A)'].values
        speed = tmp['ROTO (RPM)'].values

        # --- Estimate sample frequency (fs) ---
        dt = np.median(np.diff(time))
        if dt <= 0:
            continue
        fs = 1.0 / dt  # Hz

        current_path = os.path.join(out_dir, f"exp{exp_id}_current.png")
        speed_path = os.path.join(out_dir, f"exp{exp_id}_speed.png")
        if not os.path.exists(current_path):
            plot_spectrogram(current, fs, f"{label} Exp {exp_id} – CURRENT", current_path)
        if not os.path.exists(speed_path):
            plot_spectrogram(speed, fs, f"{label} Exp {exp_id} – SPEED", speed_path)

    print(f"Saved spectrograms to: {out_dir}\n")

In [4]:
faulty = pd.read_csv(os.path.join(INPUT_DIR, 'faulty.csv'))
healthy = pd.read_csv(os.path.join(INPUT_DIR, 'healthy.csv'))
faulty_zip = pd.read_csv(os.path.join(INPUT_DIR, 'faulty_zip.csv'))
healthy_zip = pd.read_csv(os.path.join(INPUT_DIR, 'healthy_zip.csv'))

process_dataset(faulty, "faulty")
process_dataset(healthy, "healthy")
process_dataset(faulty_zip, "faulty_zip")
process_dataset(healthy_zip, "healthy_zip")


Generating spectrograms for faulty (46 experiments)...
Saved spectrograms to: D:\Kuba\studia\Praca_magisterska\Fault-detection-in-mechanical-devices\preprocessing\..\data\preprocessed\spectrograms\faulty

Generating spectrograms for healthy (46 experiments)...
Saved spectrograms to: D:\Kuba\studia\Praca_magisterska\Fault-detection-in-mechanical-devices\preprocessing\..\data\preprocessed\spectrograms\healthy

Generating spectrograms for faulty_zip (46 experiments)...
Saved spectrograms to: D:\Kuba\studia\Praca_magisterska\Fault-detection-in-mechanical-devices\preprocessing\..\data\preprocessed\spectrograms\faulty_zip

Generating spectrograms for healthy_zip (46 experiments)...
Saved spectrograms to: D:\Kuba\studia\Praca_magisterska\Fault-detection-in-mechanical-devices\preprocessing\..\data\preprocessed\spectrograms\healthy_zip

